In [19]:
print('merge_notifica')
from sys import exit
import numpy as np
import pandas as pd
from os.path import dirname, join, isfile, isdir
from os import makedirs
from datetime import datetime, timedelta, date

from bulletin.data.notifica import Notifica
from bulletin.data.casos_confirmados import CasosConfirmados
from bulletin.commom.utils import Timer, get_better_notifica, auto_fit_columns
from bulletin.commom.static import meses
from bulletin.metabase.request import download_metabase

pd.set_option('display.max_columns', None)

output = join("output","correcoes","merge_notifica")

if not isdir(output):
    makedirs(output)

timerAll = Timer()
timer = Timer()

merge_notifica


In [2]:
cc = CasosConfirmados()
cc.load()

notifica = Notifica()
notifica.load()

print(cc.shape())
print(notifica.shape())

(588088, 10738, 432755, 144595)
(2317575, 1063803, 11045, 1240554, 2173)


In [3]:
casosc = cc.get_casos()
casosn = notifica.get_casos()
# casosn = casosn.loc[(casosn['duplicado']==0) | ((casosn['duplicado']==1) & (casosn['manter']==1))]
print(casosn.shape)

(2317575, 55)


In [4]:
idx_casos_confirmados = casosc[casosc['hash_resid'].isin(casosn['hash_resid'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_atend'].isin(casosn['hash_atend'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_diag'].isin(casosn['hash_diag'])].index.tolist()

idx_casos_confirmados = set(idx_casos_confirmados)
casos_confirmados_nao_notifica = casosc.loc[ set(casosc.index.tolist()) - idx_casos_confirmados ].sort_values('ordem')
casos_confirmados_nao_notifica.shape

(35487, 23)

In [5]:
idx_casos_notifica = casosn[casosn['hash_resid'].isin(casosc['hash_resid'])].index.tolist()
idx_casos_notifica += casosn[casosn['hash_atend'].isin(casosc['hash_atend'])].index.tolist()
idx_casos_notifica += casosn[casosn['hash_diag'].isin(casosc['hash_diag'])].index.tolist()

idx_casos_notifica = set(idx_casos_notifica)
casos_confirmados_notifica = casosn.loc[ idx_casos_notifica ].sort_values('id')
casos_confirmados_notifica.shape

(646338, 55)

In [6]:
#merge hash_resid
casos_hash_resid = casosc.loc[casosc['hash_resid'].isin(casosn['hash_resid'])]
print(f"casos_hash_resid {len(casos_hash_resid)}")
notifica_hash_resid = casosn.loc[(casosn['hash_resid'].isin(casosc['hash_resid']))]
print(f"notifica_hash_resid {len(notifica_hash_resid)}")
merge_hash_resid = pd.merge(casos_hash_resid,notifica_hash_resid[['hash_resid','id']])
print(f"merge_hash_resid {len(merge_hash_resid)}")
merge_hash_resid = merge_hash_resid.drop_duplicates('ordem')
print(f"merge_hash_resid {len(merge_hash_resid)}\n")

#merge hash_atend
casos_hash_atend = casosc.loc[casosc['hash_atend'].isin(casosn['hash_atend'])]
print(f"casos_hash_atend {len(casos_hash_atend)}")
notifica_hash_atend = casosn.loc[(casosn['hash_atend'].isin(casosc['hash_atend']))]
print(f"notifica_hash_atend {len(notifica_hash_atend)}")
merge_hash_atend = pd.merge(casos_hash_atend,notifica_hash_atend[['hash_atend','id']])
print(f"merge_hash_atend {len(merge_hash_atend)}")
merge_hash_atend = merge_hash_atend.drop_duplicates('ordem')
print(f"merge_hash_atend {len(merge_hash_atend)}\n")

#merge hash_diag
casos_hash_diag = casosc.loc[casosc['hash_diag'].isin(casosn['hash_diag'])]
print(f"casos_hash_diag {len(casos_hash_diag)}")
notifica_hash_diag = casosn.loc[(casosn['hash_diag'].isin(casosc['hash_diag']))]
print(f"notifica_hash_diag {len(notifica_hash_diag)}")
merge_hash_diag = pd.merge(casos_hash_diag,notifica_hash_diag[['hash_diag','id']])
print(f"merge_hash_diag {len(merge_hash_diag)}")
merge_hash_diag = merge_hash_diag.drop_duplicates('ordem')
print(f"merge_hash_diag {len(merge_hash_diag)}\n")

#trasnforma essa lista de index em um conjunto, para deixar apenas os index unicos
idx_casos_confirmados = set(casos_hash_resid.index.tolist() + casos_hash_atend.index.tolist() + casos_hash_diag.index.tolist())
print(f"idx_casos_confirmados {len(idx_casos_confirmados)}")

#realiza a diferença de conjunto para pegar os index dos que não foram encotrados no notifica
casos_confirmados_nao_notifica = casosc.loc[ set(casosc.index.tolist()) - idx_casos_confirmados ].sort_values('ordem')
print(f"casos_confirmados_nao_notifica {len(casos_confirmados_nao_notifica)}") ## aqui o numero total é 27910 casos não encontrados no notifica
len(idx_casos_confirmados) + len(casos_confirmados_nao_notifica)

casos_hash_resid 538780
notifica_hash_resid 633764
merge_hash_resid 633764
merge_hash_resid 538780

casos_hash_atend 508160
notifica_hash_atend 580639
merge_hash_atend 587015
merge_hash_atend 508160

casos_hash_diag 142581
notifica_hash_diag 145814
merge_hash_diag 146873
merge_hash_diag 142581

idx_casos_confirmados 552601
casos_confirmados_nao_notifica 35487


588088

In [7]:
merge_casos_notifica = pd.concat([merge_hash_resid,merge_hash_atend,merge_hash_diag,casos_confirmados_nao_notifica]).drop_duplicates('ordem').sort_values('ordem')

print(len(merge_casos_notifica))

merge_casos_notifica = pd.merge(merge_casos_notifica, casosn[['id','uf_residencia','classificacao_final','criterio_classificacao','exame','metodo','cod_evolucao','excluir_ficha','manter','duplicado']], on='id', how='left')

merge_casos_notifica.loc[merge_casos_notifica['classificacao_final'].isnull(),'classificacao_final'] = "SEM NOTIFICACAO"
merge_casos_notifica['caso'] = 1

588088


In [8]:
gcf = merge_casos_notifica.groupby('classificacao_final')[['caso','obito','recuperado','ativo']].sum()
gcf = gcf.rename(columns={'caso':'casos','obito':'obitos','recuperado':'recuperados','ativo':'ativos'})
gcf = gcf.append(pd.DataFrame(data=[gcf.sum(axis=0)],columns=gcf.columns,index=['Total']))
gcf

,casos,obitos,recuperados,ativos
CASO CONFIRMADO,504956,8951,383229,112776
CASO DESCARTADO,3294,51,694,2549
CASO SUSPEITO,44307,759,26512,17036
IGNORADO,12,0,7,5
SEM NOTIFICACAO,35487,977,22310,12200
SINDROME GRIPAL NAO ESPECIFICADA,32,0,3,29
Total,588088,10738,432755,144595


In [9]:
gce = merge_casos_notifica.groupby('cod_evolucao')[['caso','obito','recuperado','ativo']].sum()
gce = gce.rename(columns={'caso':'casos','obito':'obitos','recuperado':'recuperados','ativo':'ativos'})
gce = gce.append(pd.DataFrame(data=[gce.sum(axis=0)],columns=gce.columns,index=['Total']))
gce

,casos,obitos,recuperados,ativos
1.0,352951,547,344510,7894
2.0,8165,8056,29,80
3.0,191385,1135,65894,124356
4.0,100,23,12,65
Total,552601,9761,410445,132395


In [10]:
merge_casos_notifica = merge_casos_notifica[['ordem', 'identificacao', 'id','uf_residencia','ibge_resid', 'ibge_atend', 'paciente', 'sexo', 'idade', 'mun_resid', 'mun_atend', 'rs', 'classificacao_final', 'laboratorio', 'exame', 'metodo', 'data_diagnostico', 'data_comunicacao', 'data_1o_sintomas','cod_evolucao', 'recuperado', 'caso', 'obito', 'ativo', 'data_obito', 'data_com_obito', 'excluir','excluir_ficha','manter','duplicado']]

In [14]:
obitos_nao_notifica = merge_casos_notifica.loc[(merge_casos_notifica['obito']==1) & (merge_casos_notifica['cod_evolucao']!=2)].copy()
obitos_nao_casos_confirmados = merge_casos_notifica.loc[(merge_casos_notifica['obito']==0) & (merge_casos_notifica['cod_evolucao']==2)].copy()
recuperados_nao_notifica = merge_casos_notifica.loc[(merge_casos_notifica['recuperado']==1) & (merge_casos_notifica['cod_evolucao']!=1)].copy()
recuperados_nao_casos_confirmados = merge_casos_notifica.loc[(merge_casos_notifica['recuperado']==0) & (merge_casos_notifica['cod_evolucao']==1)].copy()

obitos_nao_notifica['diff'] = 'obitos_nao_notifica'
obitos_nao_casos_confirmados['diff'] = 'obitos_nao_casos_confirmados'
recuperados_nao_notifica['diff'] = 'recuperados_nao_notifica'
recuperados_nao_casos_confirmados['diff'] = 'recuperados_nao_casos_confirmados'

diferencas = obitos_nao_notifica.append(obitos_nao_casos_confirmados.append(recuperados_nao_notifica.append(recuperados_nao_casos_confirmados)))

gdiff = diferencas.groupby('diff')[['id']].count() #'caso','obito','recuperado','ativo'
# gdiff = gdiff.rename(columns={'caso':'casos','obito':'obitos','recuperado':'recuperados','ativo':'ativos'})
gdiff = gdiff.append(pd.DataFrame(data=[gdiff.sum(axis=0)],columns=gdiff.columns,index=['Total']))
gdiff

,id
obitos_nao_casos_confirmados,109
obitos_nao_notifica,1705
recuperados_nao_casos_confirmados,8441
recuperados_nao_notifica,65935
Total,76190


In [ ]:
casos_notifica = pd.concat([merge_hash_resid,merge_hash_atend,merge_hash_diag]).drop_duplicates('ordem').sort_values('ordem')

casos_notifica = pd.merge(merge_casos_notifica[['ordem','identificacao','id']], casosn[['id','paciente','idade','sexo','mun_resid','mun_atend']], on='id', how='left')

casos = pd.concat([casos_notifica,casos_confirmados_nao_notifica[['ordem','identificacao','id','paciente','idade','sexo','mun_resid','mun_atend']]

writer = pd.ExcelWriter(join(output,"casos_confirmados.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')


casos_confirmados.to_excel(writer,sheet_name='casos_confirmados',index=None)
worksheet = writer.sheets["casos_confirmados"]
auto_fit_columns(worksheet,casos_confirmados)

writer.save()
writer.close()

In [ ]:
recuperados = merge_casos_notifica.loc[(merge_casos_notifica['cod_evolucao']==1) & (~merge_casos_notifica['obito']==1)].copy()
writer = pd.ExcelWriter(join(output,"recuperados.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

recuperados['data'] = date.today()
recuperados['fonte'] = 'NOTIFICA'
recuperados = recuperados[['data','fonte','mun_resid','ibge_resid','paciente','sexo','idade','data_cura_obito']]

recuperados.to_excel(writer,sheet_name='recuperados',index=None)
worksheet = writer.sheets["recuperados"]
auto_fit_columns(worksheet,recuperados)

writer.save()
writer.close()

In [20]:
writer = pd.ExcelWriter(join(output,"merge_casos_notifica.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

gcf.to_excel(writer,sheet_name='totais')
worksheet = writer.sheets["totais"]
auto_fit_columns(worksheet,gcf)

merge_casos_notifica.to_excel(writer,sheet_name='geral',index=None)
worksheet = writer.sheets["geral"]
auto_fit_columns(worksheet,merge_casos_notifica)

writer.save()
writer.close()

In [21]:
writer = pd.ExcelWriter(join(output,"merge_casos_notifica_classificacao_final.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

gcf.to_excel(writer,sheet_name='totais')
worksheet = writer.sheets["totais"]
auto_fit_columns(worksheet,gcf)

for group, df in merge_casos_notifica.groupby('classificacao_final'):
    group = group[:30] if len(group) > 30 else group
    df.to_excel(writer,sheet_name=group,index=None)
    worksheet = writer.sheets[group]
    auto_fit_columns(worksheet,df)

writer.save()
writer.close()

In [22]:
writer = pd.ExcelWriter(join(output,"merge_casos_notifica_evolucao.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

gce.to_excel(writer,sheet_name='totais')
worksheet = writer.sheets["totais"]
auto_fit_columns(worksheet,gce)

for cod_evolucao, df in merge_casos_notifica.groupby('cod_evolucao'):
    df.to_excel(writer,sheet_name=f"{cod_evolucao}",index=None)
    worksheet = writer.sheets[f"{cod_evolucao}"]
    auto_fit_columns(worksheet,df)

writer.save()
writer.close()

In [23]:
writer = pd.ExcelWriter(join(output,"merge_casos_notifica_diferencas.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

gdiff.to_excel(writer,sheet_name='totais')
worksheet = writer.sheets["totais"]
auto_fit_columns(worksheet,gdiff)

for diff, df in diferencas.groupby('diff'):
    diff = diff[:30] if len(diff) > 30 else diff
    df.to_excel(writer,sheet_name=f"{diff}",index=None)
    worksheet = writer.sheets[f"{diff}"]
    auto_fit_columns(worksheet,df)

writer.save()
writer.close()